In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 KB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.4 MB/s eta 0:00:00a 0:00:01m


In [16]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 KB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 KB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 KB 11.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 KB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 KB 57.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024

In [58]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
import pickle
with open('name-to-name.pickle', 'rb') as file:
    name_to_name_dict = pickle.load(file)


def replace_symbol(text):
    text = re.sub(r'[\d′‖√∠ω№◦⊥→×]', lambda match: name_to_name_dict[match.group()], text)
    return text


def del_garbage(text):
    return text.strip('.,)?.<*&^%$#@!~')\
               .replace(u'\u00A0', ' ')\
               .replace(u'\x0c', ' ')\
               .replace('\n', ' ').replace('\t', ' ')\
               .replace('\r', ' ')


# Убираем пунтктуацию
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)



# Финальная функция обработки текста
def preprocessing_text(text):
    text = del_garbage(text).lower()
    text = replace_symbol(text)
    return text

In [2]:
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers import TrainingArguments
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

import os
import re
import numpy as np
import matplotlib.pyplot as plt
import warnings
import numpy as np
import evaluate

metric = evaluate.load("f1")
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-19 19:11:46.447720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 19:11:47.072684: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset = pd.read_csv('preprocessing_data_big.csv', sep='@')
dataset = dataset[['task', 'topic']]
dataset.rename(columns={'task': 'text',
                        'topic': 'labels'},
               inplace=True)

In [49]:
NUM_LABELS = len(dataset.labels.unique())

id2label = {id: label for id, label in enumerate(dataset.labels.unique())}

label2id = {label: id for id, label in enumerate(dataset.labels.unique())}

In [57]:
from transformers import BertForSequenceClassification
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = BertForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment',
                                                           num_labels=NUM_LABELS, id2label=id2label,
                                                           label2id=label2id,
                                                     ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at blanchefort/rubert-base-cased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
dataset["y"] = dataset.labels.map(lambda x: label2id[x])

In [10]:
dataset['text_proc'] = dataset['text'].apply(preprocessing_text)

In [11]:
dataset[dataset['text_proc'].apply(lambda x: len(x)) > 512].reset_index().text_proc[3][:512]

'одинтри. (турнир городов, дванольодинпять, восемь–девять.четыре, одинноль–одинодин.два ) на кольцевой дороге через равные промежутки расположены двапять постов, на каждом стоит полицейский. полицейские пронумерованы в какомто порядке числами от один до двапять. требуется, чтобы они перешли по дороге так, чтобы снова на каждом посту был полицейский, но по часовой стрелке за номером один стоял номер два, за номером два стоял номер три, . . . , за номером двапять стоял номер один. докажите, что если организова'

In [35]:
dataset['text_proc'] = dataset['text_proc'].apply(lambda x: x[:512])

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset['text_proc'], dataset['y'],
                                                    test_size=0.4, random_state=42,
                                                    )

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5,
                                                random_state=42)

In [15]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings  = tokenizer(list(X_val), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [25]:
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [26]:
train_dataloader = DataLoader(train_encodings, list(y_train))

val_dataloader = DataLoader(val_encodings, list(y_val))

test_dataset = DataLoader(test_encodings, list(y_test))

In [27]:
from transformers import TrainingArguments, Trainer

In [52]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [59]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [63]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./TTC4900Model', 
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0 
    num_train_epochs=20,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory                 
    logging_dir='./multi-class-logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    fp16=True,
    load_best_model_at_end=True,
)

In [64]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics=compute_metrics
)

In [65]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.559600,1.561077,0.426357,0.129058,0.110152,0.176339
100,1.551700,1.623814,0.428941,0.114711,0.092727,0.150973
150,1.469200,1.725485,0.428941,0.111615,0.090278,0.147592
200,1.605600,1.607271,0.435831,0.116384,0.093914,0.153326
250,1.620400,1.824425,0.436693,0.109400,0.090724,0.145896
300,1.630100,1.714818,0.459087,0.152476,0.117040,0.220828
350,1.589500,1.636785,0.422050,0.128727,0.111127,0.177213
400,1.510500,1.630529,0.431525,0.128368,0.106470,0.173876
450,1.470400,1.761135,0.403101,0.097105,0.084075,0.131315
500,1.438400,1.632029,0.453058,0.127804,0.103008,0.168891


Checkpoint destination directory ./TTC4900Model/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./TTC4900Model/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4360, training_loss=0.948503929650018, metrics={'train_runtime': 1401.4827, 'train_samples_per_second': 49.676, 'train_steps_per_second': 3.111, 'total_flos': 8658485142130560.0, 'train_loss': 0.948503929650018, 'epoch': 20.0})

In [80]:
q = [trainer.evaluate(eval_dataset=dataset) for dataset in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train", "val","test"]).iloc[:, :5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.837214,0.723355,0.470469,0.438131,0.519986
val,1.436659,0.530577,0.352384,0.334052,0.380386
test,1.454459,0.551249,0.362453,0.354084,0.387909


In [81]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [82]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [125]:
# Test with a an example text in Turkish
text = input()
# "Machine Learning itself is moving towards more and more automated"
predict(text)

 Максим Олегович написал на доске 2016 целых чисел. Робот Крякен заметил, что сумма любых 2015 чисел четна. Четна или нечетна сумма всех чисел?  Источник: https://shkolkovo.net/catalog/zadachi_na_teoriyu_chisel/chetnost_i_nechetnost/page-2 © shkolkovo.net


(tensor([[2.4956e-04, 1.3027e-02, 8.6578e-03, 1.9922e-02, 2.9243e-02, 5.1015e-03,
          9.2223e-01, 1.5650e-03]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(6, device='cuda:0'),
 'Теория чисел')

In [109]:
id2label

{0: 'Геометрия',
 1: 'Дирихле',
 2: 'Инвариант',
 3: 'Многочлен',
 4: 'Комбинаторика',
 5: 'Оценка+Пример',
 6: 'Теория чисел',
 7: 'Графы'}

In [119]:
df = y_train.to_frame()
df['1'] = 1
df.groupby(by=['y']).agg('count')

,1
y,
0,256
1,305
2,145
3,262
4,657
5,142
6,1493
7,221
